# USGS Stream Gage Locations

In [2]:
import requests
from datetime import datetime
import csv
import os
import pandas as pd
import numpy as np

# from google.colab import files
# files.download('example_file.txt') # specify the file path in Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def get_usgs_gage_locations(dir, data='gage height', begin_date='1950-10-01', end_date=datetime.today().strftime('%Y-%m-%d')):

  if data == 'gage height':
    url = f'https://nwis.waterdata.usgs.gov/ky/nwis/uv/?index_pmcode_00065=1&group_key=NONE&format=sitefile_output&sitefile_output_format=rdb&column_name=site_no&column_name=station_nm&column_name=dec_lat_va&column_name=dec_long_va&column_name=alt_va&column_name=huc_cd&column_name=basin_cd&column_name=rt_bol&range_selection=date_range&begin_date={begin_date}&end_date={end_date}&date_format=YYYY-MM-DD&rdb_compression=file&list_of_search_criteria=realtime_parameter_selection'
    data_path = os.path.join(dir, f'gage_height_{begin_date}_{end_date}.csv')
    metadata_path = os.path.join(dir, f'gage_height_{begin_date}_{end_date}_metadata.txt')

  elif data == 'streamflow':
    url = f'https://nwis.waterdata.usgs.gov/ky/nwis/uv/?index_pmcode_00060=1&group_key=NONE&format=sitefile_output&sitefile_output_format=rdb&column_name=site_no&column_name=station_nm&column_name=dec_lat_va&column_name=dec_long_va&column_name=alt_va&column_name=huc_cd&column_name=basin_cd&column_name=rt_bol&range_selection=date_range&begin_date={begin_date}&end_date={end_date}&date_format=YYYY-MM-DD&rdb_compression=file&list_of_search_criteria=realtime_parameter_selection'
    data_path = os.path.join(dir, f'streamflow_{begin_date}_{end_date}.csv')
    metadata_path = os.path.join(dir, f'streamflow_{begin_date}_{end_date}_metadata.txt')

  response = requests.get(url)
  text_data = response.text
  lines = text_data.splitlines()


  with open(data_path, 'w', newline='') as csvfile:

    csvwriter = csv.writer(csvfile)

    for line in lines:

      if not line.startswith('#') and line.strip():

        data = line.split('\t')
        csvwriter.writerow(data)


  with open(metadata_path, 'w') as textfile:

      for line in lines:

          # Check if the line is a comment
          if line.startswith('#'):

              # Write the comment line to the comments file
              textfile.write(line + '\n')


  return data_path

In [ ]:
gage_height_directory = f'/content/drive/MyDrive/Classes/2024_01_CS628/CS628_Project/Datasets/KYStreamGages_GageHeight'

gage_height_locations_path = get_usgs_gage_locations(gage_height_directory, data='gage height')

In [ ]:
gage_height_locations_path = r'/content/drive/MyDrive/Classes/2024_01_CS628/CS628_Project/Datasets/KYStreamGages_GageHeight/gage_height_1950-10-01_2024-02-15.csv'



df_gage_height_locations = pd.read_csv(gage_height_locations_path)

df_gage_height_locations.drop(index=0, inplace=True)

df_gage_height_locations = df_gage_height_locations.astype(dtype={'dec_lat_va':np.float32, 'dec_long_va':np.float32, 'alt_va': np.float32})

df_gage_height_locations.drop(columns=['coord_acy_cd', 'alt_acy_va', 'alt_datum_cd', 'huc_cd', 'basin_cd'], inplace=True)

df_gage_height_locations.rename(columns={'dec_lat_va':'lat', 'dec_long_va':'long', 'dec_coord_datum_cd':'datum','alt_va':'altitude', 'rt_bol':'realtime_data'}, inplace=True)

df_gage_height_locations['gage_height'] = 1

df_gage_height_locations.reset_index(drop=True, inplace=True)

df_gage_height_locations.head(10)

,site_no,station_nm,lat,long,datum,altitude,realtime_data,gage_height
0,03207965,"GRAPEVINE CREEK NEAR PHYLLIS, KY",37.432606,-82.353760,NAD83,797.190002,1,1
1,03208000,"LEVISA FORK BELOW FISHTRAP DAM NEAR MILLARD, KY",37.425938,-82.412369,NAD83,599.159973,1,1
2,03209300,"RUSSELL FORK AT ELKHORN CITY, KY",37.303997,-82.342926,NAD83,773.000000,1,1
3,03209310,"ELKHORN LAKE AT JENKINS, KY",37.168354,-82.635063,NAD83,1573.060059,NaN,1
4,03209325,"ELKHORN CREEK AT BURDINE, KY",37.187832,-82.604568,NAD83,NaN,1,1
5,03209410,"RUSSELL FORK AT CEDARVILLE, KY",37.312954,-82.359558,NAD83,743.400024,1,1
6,03209500,"LEVISA FORK AT PIKEVILLE, KY",37.464268,-82.526260,NAD83,631.260010,1,1
7,03209800,"LEVISA FORK AT PRESTONSBURG, KY",37.670929,-82.777107,NAD83,587.380005,1,1
8,03210000,"JOHNS CREEK NEAR META, KY",37.567043,-82.457924,NAD83,714.520020,1,1
9,03211500,"JOHNS CREEK NEAR VAN LEAR, KY",37.743706,-82.724045,NAD83,596.609985,1,1


In [ ]:
df_gage_height_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   site_no        251 non-null    object 
 1   station_nm     251 non-null    object 
 2   lat            251 non-null    float32
 3   long           251 non-null    float32
 4   datum          251 non-null    object 
 5   altitude       232 non-null    float32
 6   realtime_data  245 non-null    object 
 7   gage_height    251 non-null    int64  
dtypes: float32(3), int64(1), object(4)
memory usage: 12.9+ KB


In [ ]:
streamflow_directory = f'/content/drive/MyDrive/Classes/2024_01_CS628/CS628_Project/Datasets/KYStreamGages_Streamflow'

streamflow_locations_path = get_usgs_gage_locations(streamflow_directory, data='streamflow')

In [ ]:
streamflow_locations_path = r'/content/drive/MyDrive/Classes/2024_01_CS628/CS628_Project/Datasets/KYStreamGages_Streamflow/streamflow_1950-10-01_2024-02-15.csv'



df_streamflow_locations = pd.read_csv(streamflow_locations_path)

df_streamflow_locations.drop(index=0, inplace=True)

df_streamflow_locations = df_streamflow_locations.astype(dtype={'dec_lat_va':np.float32, 'dec_long_va':np.float32, 'alt_va': np.float32})

df_streamflow_locations.drop(columns=['coord_acy_cd','alt_acy_va','alt_datum_cd','huc_cd','basin_cd'], inplace=True)

df_streamflow_locations.rename(columns={'dec_lat_va':'lat', 'dec_long_va':'long', 'dec_coord_datum_cd':'datum', 'alt_va':'altitude', 'rt_bol':'realtime_data'}, inplace=True)

df_streamflow_locations['streamflow'] = 1

df_streamflow_locations.reset_index(drop=True, inplace=True)

df_streamflow_locations.head(10)

,site_no,station_nm,lat,long,datum,altitude,realtime_data,streamflow
0,03207965,"GRAPEVINE CREEK NEAR PHYLLIS, KY",37.432606,-82.353760,NAD83,797.190002,1,1
1,03209410,"RUSSELL FORK AT CEDARVILLE, KY",37.312954,-82.359558,NAD83,743.400024,1,1
2,03209500,"LEVISA FORK AT PIKEVILLE, KY",37.464268,-82.526260,NAD83,631.260010,1,1
3,03209800,"LEVISA FORK AT PRESTONSBURG, KY",37.670929,-82.777107,NAD83,587.380005,1,1
4,03210000,"JOHNS CREEK NEAR META, KY",37.567043,-82.457924,NAD83,714.520020,1,1
5,03212500,"LEVISA FORK AT PAINTSVILLE, KY",37.815372,-82.791550,NAD83,566.179993,1,1
6,03213700,"TUG FORK AT WILLIAMSON, WV",37.673157,-82.280144,NAD83,619.909973,1,1
7,03215410,"BLAINE CREEK NEAR BLAINE, KY",38.029533,-82.846832,NAD83,632.460022,1,1
8,03216070,"OHIO RIVER AT IRONTON, OH",38.532055,-82.685944,NAD83,480.410004,1,1
9,03216500,"LITTLE SANDY RIVER AT GRAYSON, KY",38.330082,-82.939339,NAD83,557.030029,1,1


In [ ]:
df_streamflow_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   site_no        227 non-null    object 
 1   station_nm     227 non-null    object 
 2   lat            227 non-null    float32
 3   long           227 non-null    float32
 4   datum          227 non-null    object 
 5   altitude       192 non-null    float32
 6   realtime_data  227 non-null    object 
 7   streamflow     227 non-null    int64  
dtypes: float32(3), int64(1), object(4)
memory usage: 11.7+ KB


# USGS Stream Gage Data

In [4]:
def get_usgs_gage_data(gage, dir, data='gage height', begin_date='1950-10-01', end_date=datetime.today().strftime('%Y-%m-%d')):

  if data == 'gage height':
    url = f'https://waterservices.usgs.gov/nwis/iv/?sites={gage}&parameterCd=00065&startDT={begin_date}T00:00:00.176-05:00&endDT={end_date}T00:00:00.176-05:00&siteStatus=all&format=rdb'
    data_path = os.path.join(dir, f'{gage}_gage_height_{begin_date}_{end_date}.csv')

  elif data == 'streamflow':
    url = f'https://waterservices.usgs.gov/nwis/iv/?sites={gage}&parameterCd=00060&startDT={begin_date}T00:00:00.176-05:00&endDT={end_date}T00:00:00.176-05:00&siteStatus=all&format=rdb'
    data_path = os.path.join(dir, f'{gage}_streamflow_{begin_date}_{end_date}.csv')

  response = requests.get(url)
  text_data = response.text
  lines = text_data.splitlines()


  with open(data_path, 'w', newline='') as csvfile:

    csvwriter = csv.writer(csvfile)

    for line in lines:

      if not line.startswith('#') and line.strip():

        data = line.split('\t')

        csvwriter.writerow(data)


  return data_path

In [ ]:
gage_height_dir = r'/content/drive/MyDrive/Classes/2024_01_CS628/CS628_Project/Datasets/KYStreamGages_GageHeight/Data'

for site in df_gage_height_locations['site_no']:

  path = get_usgs_gage_data(site, gage_height_dir)

In [ ]:
streamflow_dir = r'/content/drive/MyDrive/Classes/2024_01_CS628/CS628_Project/Datasets/KYStreamGages_Streamflow/Data'

for site in df_streamflow_locations['site_no']:

  path = get_usgs_gage_data(site, streamflow_dir, data='streamflow')